In [70]:
from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

import warnings
import numpy as np
import pandas as pd

In [71]:
plt.rc('font', family='GULIM')
warnings.filterwarnings(action='ignore')
InteractiveShell.ast_node_interactivity = "all"

# Data Check

In [72]:
wti = pd.read_csv('Database/oils/WTI.csv', index_col=0)
bno = pd.read_csv('Database/oils/BNO.csv', index_col=0)

In [73]:
wti['Adj Close']

Date
2017-12-01    3.465902
2017-12-04    3.346730
2017-12-05    3.396385
2017-12-06    3.028940
2017-12-07    3.058733
                ...   
2024-03-11    2.579681
2024-03-12    2.490040
2024-03-13    2.539840
2024-03-14    2.500000
2024-03-15    2.590000
Name: Adj Close, Length: 1581, dtype: float64

In [74]:
bno['Adj Close']

Date
2017-12-01    17.180000
2017-12-04    16.830000
2017-12-05    16.969999
2017-12-06    16.510000
2017-12-07    16.730000
                ...    
2024-03-11    29.969999
2024-03-12    29.870001
2024-03-13    30.580000
2024-03-14    31.000000
2024-03-15    31.030001
Name: Adj Close, Length: 1581, dtype: float64

# General preprocessing

In [75]:
wti['return'] = wti['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
bno['return'] = bno['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))

In [76]:
wti_vol = wti['return'].rolling(window=5).std()
bno_vol = bno['return'].rolling(window=5).std()

In [77]:
wti_vol = wti_vol.iloc[20:]
bno_vol = bno_vol.iloc[20:]

In [78]:
# test_wti = wti_vol.iloc[-52:]
# test_bno = bno_vol.iloc[-52:]

In [79]:
# train_wti = wti_vol.iloc[:-52]
# train_bno = bno_vol.iloc[:-52]

# Encoding Categories

In [80]:
wti_vol.to_csv('Database/oil_std/wti_vol.csv')
bno_vol.to_csv('Database/oil_std/bno_vol.csv')